In [43]:
import PySpin
import cv2
import numpy as np
import onnxruntime as ort

In [44]:
def configure_camera(camera):
    # Changes amount of frame captured in camera
    width = int(2448 / 1)
    height = int(2048 / 1)

    # Reset Camera
    nodemap = camera.GetNodeMap()
    user_set_selector = PySpin.CEnumerationPtr(nodemap.GetNode("UserSetSelector"))
    default_set = user_set_selector.GetEntryByName("Default")
    user_set_selector.SetIntValue(default_set.GetValue())
    user_set_load = PySpin.CCommandPtr(nodemap.GetNode("UserSetLoad"))
    user_set_load.Execute()
    
    if camera.PixelFormat.GetAccessMode() == PySpin.RW:
        camera.PixelFormat.SetValue(PySpin.PixelFormat_RGB8)
        print("Pixel format set to color.")
    else:
        print("Color pixel format not supported.")

    # Retrieve the nodemap
    nodemap = camera.GetNodeMap()

    # Get width and height nodes
    node_width = PySpin.CIntegerPtr(nodemap.GetNode("Width"))
    node_height = PySpin.CIntegerPtr(nodemap.GetNode("Height"))
    node_width_max = node_width.GetMax()
    node_height_max = node_height.GetMax()

    # Calculate offsets to center the image
    offset_x = (node_width_max - width) // 2
    offset_y = (node_height_max - height) // 2

    # Set width and height
    node_offset_x = PySpin.CIntegerPtr(nodemap.GetNode("OffsetX"))
    node_offset_y = PySpin.CIntegerPtr(nodemap.GetNode("OffsetY"))

    node_width.SetValue(width)
    node_height.SetValue(height)
    node_offset_x.SetValue(offset_x)
    node_offset_y.SetValue(offset_y)

    camera.AcquisitionMode.SetValue(PySpin.AcquisitionMode_SingleFrame)

In [54]:
def capture_and_process_image(camera, session, input_name):
    camera.BeginAcquisition()
    image_result = camera.GetNextImage()
    if image_result.IsIncomplete():
        print('Image incomplete with image status %d ...' % image_result.GetImageStatus())
        image_result.Release()
        camera.EndAcquisition()
        return True

    image_data = image_result.GetNDArray()
    image_result.Release()
    camera.EndAcquisition()
    
    # Preprocess the image for the model
    processed_image = cv2.resize(image_data, (640, 640))
    processed_image = cv2.cvtColor(processed_image, cv2.COLOR_BGR2RGB)
    processed_image = np.transpose(processed_image, (2, 0, 1)).astype(np.float32)  # HWC to CHW
    processed_image = np.expand_dims(processed_image, axis=0)  # Add batch dimension

    # Inference
    outputs = session.run(None, {input_name: processed_image})
    detections = outputs[0]  # Adjust based on your model output

    # Diagnostic: Print the structure of detections
    print("Detections:", detections)

    # Draw bounding boxes on the original image, not the processed one
    try:
        for detection in detections:
            x1, y1, x2, y2, confidence, class_id = map(int, detection[:6])
            cv2.rectangle(image_data, (x1, y1), (x2, y2), (0, 255, 0), 2)
            label = f"{class_id}: {confidence:.2f}"
            cv2.putText(image_data, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
    except Exception as e:
        print("Error processing detections:", e)
        return True

    # Display the annotated image
    cv2.imshow('Processed Image', image_data)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        return False

    return True


In [55]:
def main():
    system = PySpin.System.GetInstance()
    cam_list = system.GetCameras()
    if cam_list.GetSize() == 0:
        print('No cameras found.')
        system.ReleaseInstance()
        return

    camera = cam_list.GetByIndex(0)
    camera.Init()
    configure_camera(camera)

    # Load the ONNX model
    session = ort.InferenceSession("yolov8n.onnx")
    input_name = session.get_inputs()[0].name  # Get the input name of the model

    try:
        while True:
            if not capture_and_process_image(camera, session, input_name):
                break
    finally:
        camera.DeInit()
        del camera
        cam_list.Clear()
        system.ReleaseInstance()
        cv2.destroyAllWindows()

In [56]:
if __name__ == '__main__':
    main()

Pixel format set to color.
Detections: [[[     14.214      20.558      29.691 ...      555.57      577.45      576.97]
  [     40.075      36.555      28.678 ...      567.97      566.78      568.29]
  [     28.568      40.638      57.581 ...      168.48      123.91      125.44]
  ...
  [ 2.6315e-05  1.4901e-05  1.3381e-05 ...  5.0664e-07  4.0829e-06  1.1891e-05]
  [ 1.7285e-06  1.7583e-06  1.5199e-06 ...  3.5763e-07  6.8545e-07  1.0431e-06]
  [ 7.1824e-06  3.5763e-06  1.7881e-06 ...  1.5497e-06  6.9141e-06  1.0431e-05]]]
Error processing detections: only length-1 arrays can be converted to Python scalars
Detections: [[[     21.389      22.543      26.751 ...      549.46      578.67       593.1]
  [     36.657      11.268      16.078 ...      592.06      574.36      582.74]
  [     43.384      44.326      51.093 ...      124.47      120.31       93.19]
  ...
  [ 1.5706e-05  1.1742e-05  1.5289e-05 ...  1.8477e-06  1.1027e-05  3.6001e-05]
  [ 2.0266e-06  2.0862e-06  1.5199e-06 ...  4.1723

SpinnakerException: Spinnaker: Can't de-initialize camera. Image cleanup is required as image(s) are locked in the queue. [-1004]

In [31]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')
model.export(format = 'onnx')
onnx_model = YOLO('yolov8n.onnx')

results = onnx_model('https://ultralytics.com/images/bus.jpg')


Ultralytics YOLOv8.1.47 🚀 Python-3.10.13 torch-2.2.2 CPU (Cortex-A76)
YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs

PyTorch: starting from 'yolov8n.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 84, 8400) (6.2 MB)

ONNX: starting export with onnx 1.16.0 opset 17...
ONNX: export success ✅ 1.9s, saved as 'yolov8n.onnx' (12.2 MB)

Export complete (3.4s)
Results saved to /home/bryan/PedTracking
Predict:         yolo predict task=detect model=yolov8n.onnx imgsz=640  
Validate:        yolo val task=detect model=yolov8n.onnx imgsz=640 data=coco.yaml  
Visualize:       https://netron.app
WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.
Loading yolov8n.onnx for ONNX Runtime inference...



100%|████████████████████████████████████████| 476k/476k [00:00<00:00, 10.6MB/s]


image 1/1 /home/bryan/PedTracking/bus.jpg: 640x640 4 persons, 1 bus, 393.6ms
Speed: 10.0ms preprocess, 393.6ms inference, 2.6ms postprocess per image at shape (1, 3, 640, 640)
